#### Data Preprocessing

    In this notebook we will be doing data pre-processing. Before applying user user collaborative filtering, it's a recommended step to normalize the data, In this notebook we will be normalizing all the records and save it into sql database. 
    
    Normalization step have high complexity and takes lot of time to compute! So we will save the data into sql database


#### 1. Necessary imports

In [2]:
import numpy as np
import pandas as pd
import sqlite3 as db

#### 2. Creating SQL DB connection

In [3]:
# Connect to a database (or create one if it doesn't exist)
sql_db = 'jester_jokes'

# database location and creating sql connection!
db_loc = 'data/{}.db'.format(sql_db)
conn = db.connect(db_loc)
# Create a 'cursor' for executing commands
c = conn.cursor()

#### Extract ratings data from SQL db to pandas dataframe

In [4]:
# Selecting rating dataframe
query = 'SELECT * FROM ratings'
ratings_df = pd.read_sql(query, conn)
ratings_df.head()

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8,...,joke_91,joke_92,joke_93,joke_94,joke_95,joke_96,joke_97,joke_98,joke_99,joke_100
0,1,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,...,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,...,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,3,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,...,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,4,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,...,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,5,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,...,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


#### Step 1 : Normalization of rating around zero mean

Adjusting each user's rating vector by subtracting user's mean rating from each of there ratings(This corrects for the fact that some thinks that 5 star is anything worth seeing and other thinks 3 star is good enough)

Normalizing the ratings around 0 means that the ratings get a positive or negative feel
to them.

In [5]:
# We will be using small set of data to monitor each step and later create a method 
# r_df is subset of rating_df ! here we will only take 10 users and 8 jokes
r_df = ratings_df.iloc[0:10, 0:10]
r_df.head()

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8
0,1,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17
1,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34
2,3,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27
3,4,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21
4,5,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61


In [6]:
# Since 99 represents null value, We will replace it with blank for joke_1 column
r_df['joke_1'] = r_df['joke_1'].replace([99],np.nan)
r_df

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8
0,1,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17
1,2,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34
2,3,49,NaN,99.00,99.00,99.00,9.03,9.27,9.03,9.27
3,4,48,NaN,8.35,99.00,99.00,1.80,8.16,-2.82,6.21
4,5,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61
5,6,100,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87
6,7,47,NaN,99.00,99.00,99.00,8.59,-9.85,7.72,8.79
7,8,100,6.84,3.16,9.17,-6.21,-8.16,-1.70,9.27,1.41
8,9,100,-3.79,-3.54,-9.42,-6.89,-8.74,-0.29,-5.29,-8.93
9,10,72,3.01,5.15,5.15,3.01,6.41,5.15,8.93,2.52


In [7]:
r_df.iloc[1, 2:] - np.mean(r_df.iloc[1, 2:])

joke_1    4.52875
joke_2    0.15875
joke_3    6.80875
joke_4    4.81875
joke_5   -1.93125
joke_6   -9.21125
joke_7   -0.28125
joke_8   -4.89125
Name: 1, dtype: float64

In [8]:
# Selecting all joke columns
joke_columns = r_df.columns[2:] # joke column starts from second column
joke_columns

Index(['joke_1', 'joke_2', 'joke_3', 'joke_4', 'joke_5', 'joke_6', 'joke_7',
       'joke_8'],
      dtype='object')

#### Creating the method for normalization as learnt in the tutorial above!

In [11]:
# replacing 99 with nan value
def replace_99(ratings):
    '''
    Replace 99 with nan for user's rating value
    step 1: select joke columns
    step 2: replace 99 with np.nan for all columns
    
    '''
    joke_ids = ratings.columns[2:]    
    for joke_id in joke_ids: 
        ratings[joke_id] = ratings[joke_id].replace([99],np.nan)
    return ratings

# Now creating a function that takes all the raw data as input and complete step 1 
def normalization(ratings):
    ''' 
    Returns mean subtracted user's rating value
    step 1: subtract user's rating by their mean value for each row 
    '''
    total_users = ratings.shape[0]
    for i in range(total_users):
        ratings.iloc[i, 2:] = ratings.iloc[i, 2:] - np.mean(ratings.iloc[i, 2:])
    return ratings

# replacing nan with 0
def replace_0(ratings):
    '''
    Replace nan with 0 for user's rating value
    step 1: select joke columns
    step 2: replace np.nan with 0 for all columns
    
    '''
    joke_ids = ratings.columns[2:]    
    for joke_id in joke_ids: 
        ratings[joke_id] = ratings[joke_id].replace([np.nan],0)
    return ratings

In [13]:
## Here Spark will be better choce that
temp1 = replace_99(r_df)
temp2 = normalization(temp1)
temp3 = replace_0(temp2)
temp3.head()

,user_id,number_of_jokes_rated,joke_1,joke_2,joke_3,joke_4,joke_5,joke_6,joke_7,joke_8
0,1,74,-3.001250e+00,1.360875e+01,-4.841250e+00,-3.341250e+00,-2.70125,-3.68125,-5.03125,8.98875
1,2,100,4.528750e+00,1.587500e-01,6.808750e+00,4.818750e+00,-1.93125,-9.21125,-0.28125,-4.89125
2,3,49,-4.440892e-16,-4.440892e-16,-4.440892e-16,-4.440892e-16,-0.12000,0.12000,-0.12000,0.12000
3,4,48,6.106227e-16,4.010000e+00,6.106227e-16,6.106227e-16,-2.54000,3.82000,-7.16000,1.87000
4,5,91,6.230000e+00,2.340000e+00,-6.440000e+00,-7.660000e+00,-0.91000,-0.67000,4.77000,2.34000


#### Now transformation of rating data for into normalized ratings for all the ratings! 

In [ ]:
# we will normalize and save into normalized_ratings dataframe!
normalized_ratings = replace_99(ratings_df)
normalized_ratings = normalization(normalized_ratings)
normalized_ratings = replace_0(normalized_ratings)
normalized_ratings.head()

# Saving normalized ratings_df into sql database
# because computationally it takes lot of time
schema = "CREATE TABLE normalized_ratings(user_id Integer, number_of_jokes_rated Integer,"
for i in range(1, 101):
    schema = schema + 'joke_{} FLOAT(2), '.format(str(i))
schema = schema[:-2] + ')'
c.execute(schema)

query =  'INSERT INTO normalized_ratings VALUES('
for i in range(0, 102):
    query = query + '?,'
query = query[:-1] + ')'
# saving normalized ratings data
c.executemany(query, normalized_ratings.values)
# Save (commit) the changes
conn.commit()

## Finding similarity: 
We will be using pearson correlation as similarity measure !

In [17]:
ratings_of_active_user = temp3.iloc[6, 2:]
ratings_of_other_user = temp3.iloc[7, 2:]

In [18]:
# note probably use list and assert that x and y are list!
# since we are using normalized data, we won't subtract by mean value again
def weight_factor(x, y):
    ''' 
    Weight factor implies relationship between user x and user y
    Also know as similarity between user x and user y
    We are using Pearson correlation coefficient here. 
    '''    
    t1, t2, t3 = 0, 0, 0 
    for i, j in zip(x, y):
        t1+=i*j
        t2+=i*i
        t3+=j*j
    return t1/(np.sqrt(t2) * np.sqrt(t3))

    Testing out similarity 

In [20]:
# now our active user is suppose user_id 1!
# we want to find similarity with other users!

# x represent active user
# y represents other user
x =  temp3.iloc[0, 2:]
similarity = [weight_factor(x, temp3.iloc[i, 2:]) for i in range(temp3.shape[0])]
similarity

[1.0,
 -0.23803270911260682,
 0.34567259955147395,
 0.5542047700379624,
 0.2889392861822395,
 0.398903352140551,
 0.20967057910055695,
 -0.020480436945635686,
 0.017735643966224027,
 -0.3068296372701195]

----------------------------------------------------------------
                                        END
                                        
-----------------------------------------------------------------